# Simulation 

### Import modules

In [1]:
import numpy as np
import random 
import simpy
import time
import csv
from collections import namedtuple
from datetime import datetime
from datetime import timedelta
from scipy.interpolate import interp1d

### Functions for the simulations

In [2]:
def imps():
    """Funtions that generates the number of impressions for a br
    
    """
    lam = int(np.random.normal(loc=4, scale=2, size=1))
    if lam < 1:
        lam = 1
    nb_imp = np.random.poisson(lam)
    return nb_imp

In [3]:
def delay(lam):
    """Function that generates a number of seconds before the next br following a Poisson law 
    of lambda parameter.
    
    Arguments:
    :lam: expected number of seconds 
    """
    seconds = np.random.poisson(lam)
    # 1% probability of technical error
    if not random.random() < 0.99:
        seconds = np.random.poisson(seconds + 1000)
    return seconds

In [4]:
def saving(data, filename):
    """Function to save a named tuple to CSV
    
    Arguments:
    :data: A list of named tuple to transform into CSV
    :filename: Name of the file to create
    """
    with open(filename, "w", encoding="utf8") as file:
        #Column name for first row
        first, *_ = data
        writer = csv.DictWriter(file, first._fields)
        writer.writeheader()
        for br in data:
            writer.writerow(br._asdict())

### Simulation function

In [5]:
def open_rtb(env, P, timestampnow, nb_days, nb_hours_per_day, bidrequests, data):
    """Function to simulate BR
    
    Arguments:
    :env: A Simpy environment
    :P: Fix price of 1 impression
    :timestampnow: Timestamp when the simulation starts
    :nb_days: Number of days to simulate
    :nb_hours_per_days: Number of opening hours of the br
    :bidrequests: A named tuple to store data
    :data: An empty list
    """
    day = datetime.now().day
    month = datetime.now().month
    year = datetime.now().year
    next_date = datetime(year,month,day,6,0,0,0)
    setup = True
    ID = 0
    while True:
        # Are we in opening hours?
        current_hour = datetime.fromtimestamp(env.now).hour
        if current_hour >= 6 and current_hour < 20:
            if setup:
                setup = False
                next_date += timedelta(days=1)
                lambdas_limits = np.array([1,60,30,5])
                timestamps_limits = np.array([env.now,env.now+32400,env.now+39600,env.now+54000])
                f_lambdas = interp1d(timestamps_limits, lambdas_limits)
                
            # Generate a br
            ID += 1
            
            # Timestamp of br
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")

            # Number of impressions
            nb_imp = imps()
            price = P * nb_imp
            
            # Win/loose and number of seconds before the notification
            seconds_notif = random.randint(2,900)
            if not random.random() < 0.95:
                win = False
            else:
                win = True

            # Storing data
            results = bidrequests(
                ID = ID,
                timestamp = env.now,
                timestamp_string = time,
                nb_imp = nb_imp,
                price = price,
                win = win,
                seconds_notif = seconds_notif
            )
            data.append(results)

            # Time before next BR
            time_before_next = delay(f_lambdas(env.now))
            
            # Remaining time before the end of the simulation
            rt = timestampnow + timedelta(days = nb_days).total_seconds() - env.now

            # End of the simulation
            if rt < time_before_next:
                print(f"End of simulation at {datetime.fromtimestamp(env.now)}")

            yield env.timeout(time_before_next)

        else:
            time_to_wait = datetime.timestamp(next_date) - env.now
            setup = True
            current_hour = datetime.fromtimestamp(env.now).hour
            rt = timestampnow + timedelta(days = nb_days).total_seconds() - env.now
            if rt <= time_to_wait:
                print(f"End of simulation at {datetime.fromtimestamp(env.now)}")
            yield env.timeout(time_to_wait) 

In [6]:
timestampnow = datetime.timestamp(datetime(2020,7,9,6,0,0,0))
bidrequests =  namedtuple(
    "bidrequests", 
    (
        "ID", 
        "timestamp", 
        "timestamp_string",
        "nb_imp",
        "price",
        "win",
        "seconds_notif"
    )
                    )
data = list()
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb(env = env, P = 1, timestampnow = timestampnow, 
                                               nb_days = 30, nb_hours_per_day = 14, 
                                               bidrequests = bidrequests, data = data))
env.run(until=timestampnow + timedelta(days = 30).total_seconds())

End of simulation at 2020-08-07 20:00:04


In [8]:
saving(data, '09-07-2020_07-08-2020.csv')